This is a record of all the code used to compute summary stats.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from numba import njit, prange
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib as mpl
import pandas as pd
from sklearn.multiclass import OneVsRestClassifier as classifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
from sklearn.metrics import log_loss, roc_curve
import optuna
from sklearn.cluster import AgglomerativeClustering as clustering

base_filepath = "/Users/44749/Documents/PhD_work/datasets_for_analysis/"

In [2]:

#choose samples, making sure they're not too close together
@njit()
def choose_samples(sample_locs, no_samples):
    chosen_indices, chosen_locs = [0], [sample_locs[0]] #take the first sample always, assume a point chosen at random
    #of the available samples, take the one that maximises the total distance from all current samples
    while len(chosen_indices) < no_samples:
        furthest_current_index, max_total_distance = 0, 0
        for num, loc in enumerate(sample_locs):
            if num not in chosen_indices:
                total_distance = sum([np.sqrt((loc[0]-x)**2 + (loc[1]-y)**2 + (loc[2]-z)**2) for [x, y, z] in chosen_locs])
                if total_distance > max_total_distance:
                    max_total_distance = total_distance
                    furthest_current_index = num
        chosen_indices.append(furthest_current_index)
        chosen_locs.append(sample_locs[furthest_current_index])
    return chosen_indices

#calculate the number of mutations fixed in n samples
@njit()
def count_fixation_number(mut_matrix, num_samples):
    number_fixed = 0
    for row in mut_matrix:
        num_samples_fixed = 0
        for read in row:
            if read == 1.0:
                num_samples_fixed += 1
        if num_samples_fixed == num_samples:
            number_fixed += 1
    return number_fixed


prim_read_threshold = 10/160
second_read_threshold = 4/160

#calculate output summary statistics for a given number of samples, excluding clonal mutations
def calculate_output_stats(filename, prefix, base_filepath, number, no_samples):
    #load files
    mutdict = np.load(base_filepath+filename+"/"+prefix+"_"+str(number)+"_mutdict.npy", allow_pickle=True)
    sample_locs = np.load(base_filepath+filename+"/"+prefix+"_"+str(number)+"_sample_locs.npy", allow_pickle=True)
    #to mimic actual sampling procedure
    chosen_indices = choose_samples(sample_locs, no_samples)
    sample_dicts = [mutdict[index] for index in chosen_indices]
    #look at all mutations seen
    all_muts = []
    for dict in sample_dicts:
        all_muts += list(dict.keys()) #look at all mutations seen in this sample
    all_unique_mutations = np.unique(all_muts) #all mutations in all samples
    mut_matrix = np.zeros((len(all_unique_mutations), no_samples))
    for m, mut in enumerate(all_unique_mutations):
        for n, dict in enumerate(sample_dicts):
            if mut in dict:
                mut_matrix[m][n] = dict[mut] #zero by default
    min_ccfs = np.min(mut_matrix, axis=1)
    non_clonal_muts = np.where(min_ccfs < 1.0)[0] #not clonal everywhere
    mut_matrix = mut_matrix[non_clonal_muts] #filter out all of the clonal ones
    #we need it to be detected above 10 reads somewhere; 4 reads in the relevant sample is already built in
    max_ccfs = np.max(mut_matrix, axis=1)
    sufficient = np.where(max_ccfs >= prim_read_threshold)[0]
    mut_matrix = mut_matrix[sufficient]
    #now to actually calculate summary statistics!
    #0. all mutations detected subclonally (i.e. with CCF < 1.0 in at least one sample)
    no_muts = len(mut_matrix)
    #1. all mutations detected in 1 sample only
    appearances = np.count_nonzero(mut_matrix, axis=1)
    private = np.where(appearances == 1)[0]
    num_private = len(private)
    #2-no_samples, ... number fixed in 1, ... no_samples-1 samples
    fixed_muts = []
    for s in range(1, no_samples):
        fixed_muts.append(count_fixation_number(mut_matrix, s)) #count number fixed in s samples- here it MAY THEN APPEAR IN A NUMBER OF OTHER SAMPLES, JUST NOT FIXED
    #no_samples+1: number of mutations fixed AND private
    private_ccfs = np.max(mut_matrix[private], axis=1) #the maximum CCF of a private mutation will be its CCF in that sample
    fixed_and_private = np.where(private_ccfs == 1.0)[0]
    num_fixed_private = len(fixed_and_private)
    #no_samples + 2-2*no_samples-1- the number of mutations appearing in 2, ... no_samples samples
    num_appearing = []
    for s in range(2, no_samples+1):
        num_appearing.append(len(np.where(appearances == s)[0]))
    return [no_muts, num_private] + fixed_muts + [num_fixed_private] + num_appearing    
    
    

For full run of tumours at mutation rate 0.6 per cell division.

In [6]:
filenames = ["27Jul_gen_0.6"]*3
prefixes = ["nosel", "lowsel", "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples nosel
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 762 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 1258 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 2075 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 3190 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 3365 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 3657 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 4595 failed
For 3 samples nosel
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 762 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 1258 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 2075 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 3190 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 3365 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 3657 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 4595 failed
For 4 samples nosel
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 762 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 1258 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 2075 failed
27Jul_gen_0.6 nosel 27Jul_gen_0.6_nosel 3190 failed
27Jul_g

In [7]:
filenames = ["27Jul_1.0_0.6"]*2
prefixes = ["lowsel", "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]




max_counts = 5000



for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
For 3 samples lowsel
For 4 samples lowsel
For 5 samples lowsel
For 6 samples lowsel
For 7 samples lowsel
For 8 samples lowsel
For 2 samples highsel
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 313 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 350 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 409 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 478 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 661 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 772 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 949 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 951 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 1253 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 1624 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 1703 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 1750 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 1889 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 1967 failed
27Jul_1.0_0.6 highsel 27Jul_1.0_0.6_highsel 2091 failed

In [8]:
filenames = ["27Jul_0.98_0.6"]*2
prefixes = ["lowsel", "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 6 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 25 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 75 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 152 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 181 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 183 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 223 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 247 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 257 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 258 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 268 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 273 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 286 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 388 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 469 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 628 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 638 failed
27Jul_0.98_0.6 lowsel 27Jul_0.98_0.6_lowsel 643 

In [3]:
filenames = ["27Jul_0.95_0.6"]*1
prefixes = ['highsel'] #["lowsel", "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples highsel
For 3 samples highsel
For 4 samples highsel
For 5 samples highsel
For 6 samples highsel
For 7 samples highsel
For 8 samples highsel


In [4]:
filenames = ["27Jul_0.9_0.6"]*1
prefixes = ["lowsel"] #"highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 10 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 39 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 40 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 84 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 92 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 172 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 174 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 178 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 210 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 211 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 222 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 228 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 232 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 246 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 270 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 292 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 300 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9_0.6_lowsel 322 failed
27Jul_0.9_0.6 lowsel 27Jul_0.9

In [11]:
filenames = ["27Jul_0.75_0.6"]*2
prefixes = ["lowsel", "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 28 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 61 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 70 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 77 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 85 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 120 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 199 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 216 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 221 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 273 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 280 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 320 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 357 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 370 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 371 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 417 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 422 failed
27Jul_0.75_0.6 lowsel 27Jul_0.75_0.6_lowsel 491 f

In [12]:
filenames = ["27Jul_0.5_0.6"]*2
prefixes = ["lowsel", "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 6 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 18 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 22 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 34 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 35 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 46 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 67 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 74 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 76 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 92 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 112 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 120 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 121 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 146 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 159 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 164 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 175 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_lowsel 183 failed
27Jul_0.5_0.6 lowsel 27Jul_0.5_0.6_l

In [7]:
filenames = ["27Jul_0.0_0.6"]*2
prefixes = ["lowsel", "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 25 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 33 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 53 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 110 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 128 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 136 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 169 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 197 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 247 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 270 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 281 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 297 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 335 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 345 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 346 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 363 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 413 failed
27Jul_0.0_0.6 lowsel 27Jul_0.0_0.6_lowsel 416 failed
27Jul_0.0_0.6 lowsel 27Jul_0

In [6]:
filenames = ["27Jul_1.0_0.4"]*2
prefixes = ["lowsel", "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
For 3 samples lowsel
For 4 samples lowsel
For 5 samples lowsel
For 6 samples lowsel
For 7 samples lowsel
For 8 samples lowsel
For 2 samples highsel
For 3 samples highsel
For 4 samples highsel
For 5 samples highsel
For 6 samples highsel
For 7 samples highsel
For 8 samples highsel


In [5]:
filenames = ["27Jul_gen_0.4"]*3
prefixes = ["nosel", "lowsel", "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples nosel
For 3 samples nosel
For 4 samples nosel
For 5 samples nosel
For 6 samples nosel
For 7 samples nosel
For 8 samples nosel
For 2 samples lowsel
For 3 samples lowsel
For 4 samples lowsel
For 5 samples lowsel
For 6 samples lowsel
For 7 samples lowsel
For 8 samples lowsel
For 2 samples highsel
27Jul_gen_0.4 highsel 27Jul_gen_0.4_highsel 4121 failed
27Jul_gen_0.4 highsel 27Jul_gen_0.4_highsel 4122 failed
27Jul_gen_0.4 highsel 27Jul_gen_0.4_highsel 4123 failed
27Jul_gen_0.4 highsel 27Jul_gen_0.4_highsel 4124 failed
27Jul_gen_0.4 highsel 27Jul_gen_0.4_highsel 4125 failed
27Jul_gen_0.4 highsel 27Jul_gen_0.4_highsel 4126 failed
For 3 samples highsel
27Jul_gen_0.4 highsel 27Jul_gen_0.4_highsel 4121 failed
27Jul_gen_0.4 highsel 27Jul_gen_0.4_highsel 4122 failed
27Jul_gen_0.4 highsel 27Jul_gen_0.4_highsel 4123 failed
27Jul_gen_0.4 highsel 27Jul_gen_0.4_highsel 4124 failed
27Jul_gen_0.4 highsel 27Jul_gen_0.4_highsel 4125 failed
27Jul_gen_0.4 highsel 27Jul_gen_0.4_highsel 4126 fail

In [6]:
filenames = ["27Jul_0.98_0.4"]*1
prefixes = ["lowsel"]# "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
For 3 samples lowsel
For 4 samples lowsel
For 5 samples lowsel
For 6 samples lowsel
For 7 samples lowsel
For 8 samples lowsel


In [4]:
filenames = ["27Jul_0.95_0.4"]*1
prefixes = ["lowsel"]# "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
For 3 samples lowsel
For 4 samples lowsel
For 5 samples lowsel
For 6 samples lowsel
For 7 samples lowsel
For 8 samples lowsel


In [7]:
filenames = ["27Jul_0.9_0.4"]*1
prefixes = ["lowsel"]# "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
For 3 samples lowsel
For 4 samples lowsel
For 5 samples lowsel
For 6 samples lowsel
For 7 samples lowsel
For 8 samples lowsel


In [8]:
filenames = ["27Jul_0.75_0.4"]*2
prefixes = ["lowsel", "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
For 3 samples lowsel
For 4 samples lowsel
For 5 samples lowsel
For 6 samples lowsel
For 7 samples lowsel
For 8 samples lowsel
For 2 samples highsel
For 3 samples highsel
For 4 samples highsel
For 5 samples highsel
For 6 samples highsel
For 7 samples highsel
For 8 samples highsel


In [9]:
filenames = ["27Jul_0.5_0.4"]*2
prefixes = ["lowsel", "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 3525 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 3970 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 4350 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 4547 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 4851 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 4870 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 4875 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 4884 failed
For 3 samples lowsel
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 3525 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 3970 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 4350 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 4547 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 4851 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 4870 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 4875 failed
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 4884 failed
For 4 samples lowsel
27Jul_0.5_0.4 lowsel 27Jul_0.5_0.4_lowsel 3525 failed
27Jul_0.5_0.4 lowse

In [3]:
filenames = ["27Jul_0.0_0.4"]*2
prefixes = ["lowsel", "highsel"]
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5000

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
For 3 samples lowsel
For 4 samples lowsel
For 5 samples lowsel
For 6 samples lowsel
For 7 samples lowsel
For 8 samples lowsel
For 2 samples highsel
For 3 samples highsel
For 4 samples highsel
For 5 samples highsel
For 6 samples highsel
For 7 samples highsel
For 8 samples highsel


Now the overlay simulations, with genetic and nongenetic selection.

In [7]:
filenames = ["1Aug_1.0_" + suff for suff in ["strength_1", "strength_2", "strength_5", "freq_10", "freq_100", "freq_1000"]]
used_filenames = [f+"/"+f for f in filenames]
prefixes = ["lowsel"]*6
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5005

for (filename, prefix, matrix_name) in zip(used_filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
1Aug_1.0_strength_1/1Aug_1.0_strength_1 lowsel 1Aug_1.0_strength_1_lowsel 1 failed
1Aug_1.0_strength_1/1Aug_1.0_strength_1 lowsel 1Aug_1.0_strength_1_lowsel 2 failed
1Aug_1.0_strength_1/1Aug_1.0_strength_1 lowsel 1Aug_1.0_strength_1_lowsel 3 failed
1Aug_1.0_strength_1/1Aug_1.0_strength_1 lowsel 1Aug_1.0_strength_1_lowsel 4 failed
1Aug_1.0_strength_1/1Aug_1.0_strength_1 lowsel 1Aug_1.0_strength_1_lowsel 5 failed
For 3 samples lowsel
1Aug_1.0_strength_1/1Aug_1.0_strength_1 lowsel 1Aug_1.0_strength_1_lowsel 1 failed
1Aug_1.0_strength_1/1Aug_1.0_strength_1 lowsel 1Aug_1.0_strength_1_lowsel 2 failed
1Aug_1.0_strength_1/1Aug_1.0_strength_1 lowsel 1Aug_1.0_strength_1_lowsel 3 failed
1Aug_1.0_strength_1/1Aug_1.0_strength_1 lowsel 1Aug_1.0_strength_1_lowsel 4 failed
1Aug_1.0_strength_1/1Aug_1.0_strength_1 lowsel 1Aug_1.0_strength_1_lowsel 5 failed
For 4 samples lowsel
1Aug_1.0_strength_1/1Aug_1.0_strength_1 lowsel 1Aug_1.0_strength_1_lowsel 1 failed
1Aug_1.0_strength_1/1Aug

In [6]:
filenames = ["1Aug_0.98_" + suff for suff in ["strength_1", "strength_2", "strength_5", "freq_10", "freq_100", "freq_1000"]]
used_filenames = [f+"/"+f for f in filenames]
prefixes = ["lowsel"]*6
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5005

for (filename, prefix, matrix_name) in zip(used_filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
1Aug_0.98_strength_1/1Aug_0.98_strength_1 lowsel 1Aug_0.98_strength_1_lowsel 1 failed
1Aug_0.98_strength_1/1Aug_0.98_strength_1 lowsel 1Aug_0.98_strength_1_lowsel 2 failed
1Aug_0.98_strength_1/1Aug_0.98_strength_1 lowsel 1Aug_0.98_strength_1_lowsel 3 failed
1Aug_0.98_strength_1/1Aug_0.98_strength_1 lowsel 1Aug_0.98_strength_1_lowsel 4 failed
1Aug_0.98_strength_1/1Aug_0.98_strength_1 lowsel 1Aug_0.98_strength_1_lowsel 5 failed
1Aug_0.98_strength_1/1Aug_0.98_strength_1 lowsel 1Aug_0.98_strength_1_lowsel 3482 failed
1Aug_0.98_strength_1/1Aug_0.98_strength_1 lowsel 1Aug_0.98_strength_1_lowsel 3837 failed
1Aug_0.98_strength_1/1Aug_0.98_strength_1 lowsel 1Aug_0.98_strength_1_lowsel 3976 failed
1Aug_0.98_strength_1/1Aug_0.98_strength_1 lowsel 1Aug_0.98_strength_1_lowsel 3977 failed
1Aug_0.98_strength_1/1Aug_0.98_strength_1 lowsel 1Aug_0.98_strength_1_lowsel 3996 failed
1Aug_0.98_strength_1/1Aug_0.98_strength_1 lowsel 1Aug_0.98_strength_1_lowsel 4011 failed
1Aug_0.98_stren

In [5]:
filenames = ["1Aug_0.95_" + suff for suff in ["strength_1", "strength_2", "strength_5", "freq_10", "freq_100", "freq_1000"]]
used_filenames = [f+"/"+f for f in filenames]
prefixes = ["lowsel"]*6
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5005

for (filename, prefix, matrix_name) in zip(used_filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
1Aug_0.95_strength_1/1Aug_0.95_strength_1 lowsel 1Aug_0.95_strength_1_lowsel 1 failed
1Aug_0.95_strength_1/1Aug_0.95_strength_1 lowsel 1Aug_0.95_strength_1_lowsel 2 failed
1Aug_0.95_strength_1/1Aug_0.95_strength_1 lowsel 1Aug_0.95_strength_1_lowsel 3 failed
1Aug_0.95_strength_1/1Aug_0.95_strength_1 lowsel 1Aug_0.95_strength_1_lowsel 4 failed
1Aug_0.95_strength_1/1Aug_0.95_strength_1 lowsel 1Aug_0.95_strength_1_lowsel 5 failed
1Aug_0.95_strength_1/1Aug_0.95_strength_1 lowsel 1Aug_0.95_strength_1_lowsel 126 failed
1Aug_0.95_strength_1/1Aug_0.95_strength_1 lowsel 1Aug_0.95_strength_1_lowsel 161 failed
1Aug_0.95_strength_1/1Aug_0.95_strength_1 lowsel 1Aug_0.95_strength_1_lowsel 166 failed
1Aug_0.95_strength_1/1Aug_0.95_strength_1 lowsel 1Aug_0.95_strength_1_lowsel 171 failed
1Aug_0.95_strength_1/1Aug_0.95_strength_1 lowsel 1Aug_0.95_strength_1_lowsel 176 failed
1Aug_0.95_strength_1/1Aug_0.95_strength_1 lowsel 1Aug_0.95_strength_1_lowsel 271 failed
1Aug_0.95_strength_1/

In [4]:
filenames = ["1Aug_0.9_" + suff for suff in ["strength_1", "strength_2", "strength_5", "freq_10", "freq_100", "freq_1000"]]
used_filenames = [f+"/"+f for f in filenames]
prefixes = ["lowsel"]*6
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5005

for (filename, prefix, matrix_name) in zip(used_filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
1Aug_0.9_strength_1/1Aug_0.9_strength_1 lowsel 1Aug_0.9_strength_1_lowsel 1 failed
1Aug_0.9_strength_1/1Aug_0.9_strength_1 lowsel 1Aug_0.9_strength_1_lowsel 2 failed
1Aug_0.9_strength_1/1Aug_0.9_strength_1 lowsel 1Aug_0.9_strength_1_lowsel 3 failed
1Aug_0.9_strength_1/1Aug_0.9_strength_1 lowsel 1Aug_0.9_strength_1_lowsel 4 failed
1Aug_0.9_strength_1/1Aug_0.9_strength_1 lowsel 1Aug_0.9_strength_1_lowsel 5 failed
1Aug_0.9_strength_1/1Aug_0.9_strength_1 lowsel 1Aug_0.9_strength_1_lowsel 2460 failed
1Aug_0.9_strength_1/1Aug_0.9_strength_1 lowsel 1Aug_0.9_strength_1_lowsel 2589 failed
1Aug_0.9_strength_1/1Aug_0.9_strength_1 lowsel 1Aug_0.9_strength_1_lowsel 2590 failed
1Aug_0.9_strength_1/1Aug_0.9_strength_1 lowsel 1Aug_0.9_strength_1_lowsel 3344 failed
1Aug_0.9_strength_1/1Aug_0.9_strength_1 lowsel 1Aug_0.9_strength_1_lowsel 3345 failed
For 3 samples lowsel
1Aug_0.9_strength_1/1Aug_0.9_strength_1 lowsel 1Aug_0.9_strength_1_lowsel 1 failed
1Aug_0.9_strength_1/1Aug_0.9_s

In [4]:
filenames = ["1Aug_0.75_" + suff for suff in ["strength_1", "strength_2", "strength_5", "freq_10", "freq_100", "freq_1000"]]
#used_filenames = [f+"/"+f for f in filenames]
prefixes = ["lowsel"]*6
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5005

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename+"/"+filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        #print(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 1 failed
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 2 failed
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 3 failed
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 4 failed
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 5 failed
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 649 failed
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 1549 failed
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 2459 failed
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 3185 failed
For 3 samples lowsel
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 1 failed
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 2 failed
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 3 failed
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 4 failed
1Aug_0.75_strength_1 lowsel 1Aug_0.75_strength_1_lowsel 5 failed
1Aug_0.75_strength_1 lowsel 1Aug_0.75

In [5]:
filenames = ["1Aug_0.5_" + suff for suff in ["strength_1", "strength_2", "strength_5", "freq_10", "freq_100", "freq_1000"]]
#filenames = [f+"/"+f for f in filenames]
prefixes = ["lowsel"]*6
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5005

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename+"/"+filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 1 failed
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 2 failed
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 3 failed
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 4 failed
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 5 failed
For 3 samples lowsel
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 1 failed
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 2 failed
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 3 failed
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 4 failed
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 5 failed
For 4 samples lowsel
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 1 failed
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 2 failed
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 3 failed
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 4 failed
1Aug_0.5_strength_1 lowsel 1Aug_0.5_strength_1_lowsel 5

In [6]:
filenames = ["1Aug_0.0_" + suff for suff in ["strength_1", "strength_2", "strength_5", "freq_10", "freq_100", "freq_1000"]]
#filenames = [f+"/"+f for f in filenames]
prefixes = ["lowsel"]*6
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 5005

for (filename, prefix, matrix_name) in zip(filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename+"/"+filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples lowsel
For 3 samples lowsel
For 4 samples lowsel
For 5 samples lowsel
For 6 samples lowsel
For 7 samples lowsel
For 8 samples lowsel
For 2 samples lowsel
For 3 samples lowsel
For 4 samples lowsel
For 5 samples lowsel
For 6 samples lowsel
For 7 samples lowsel
For 8 samples lowsel
For 2 samples lowsel
1Aug_0.0_strength_5 lowsel 1Aug_0.0_strength_5_lowsel 1 failed
1Aug_0.0_strength_5 lowsel 1Aug_0.0_strength_5_lowsel 2 failed
1Aug_0.0_strength_5 lowsel 1Aug_0.0_strength_5_lowsel 3 failed
1Aug_0.0_strength_5 lowsel 1Aug_0.0_strength_5_lowsel 4 failed
1Aug_0.0_strength_5 lowsel 1Aug_0.0_strength_5_lowsel 5 failed
For 3 samples lowsel
1Aug_0.0_strength_5 lowsel 1Aug_0.0_strength_5_lowsel 1 failed
1Aug_0.0_strength_5 lowsel 1Aug_0.0_strength_5_lowsel 2 failed
1Aug_0.0_strength_5 lowsel 1Aug_0.0_strength_5_lowsel 3 failed
1Aug_0.0_strength_5 lowsel 1Aug_0.0_strength_5_lowsel 4 failed
1Aug_0.0_strength_5 lowsel 1Aug_0.0_strength_5_lowsel 5 failed
For 4 samples lowsel
1Aug_0.0_stre

Now look at size, time, deme and replacement-rate tests.

In [10]:
filenames = ["22Jul_2m", "22Jul_10m", "22Jul_20m"]*3
used_filenames = ["22Jul_tests_12Aug"+"/"+f for f in filenames]
prefixes = ["nosel"]*3 + ["lowsel"]*3 + ["highsel"]*3
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 205

for (filename, prefix, matrix_name) in zip(used_filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples nosel
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 1 failed
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 202 failed
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 203 failed
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 204 failed
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 205 failed
For 3 samples nosel
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 1 failed
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 202 failed
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 203 failed
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 204 failed
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 205 failed
For 4 samples nosel
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 1 failed
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 202 failed
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 203 failed
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 204 failed
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m_nosel 205 failed
For 5 samples nosel
22Jul_tests_12Aug/22Jul_2m nosel 22Jul_2m

In [11]:
filenames = ["22Jul_6mo", "22Jul_18mo", "22Jul_2y"]*3
used_filenames = ["22Jul_tests_12Aug"+"/"+f for f in filenames]
prefixes = ["nosel"]*3 + ["lowsel"]*3 + ["highsel"]*3
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 205

for (filename, prefix, matrix_name) in zip(used_filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples nosel
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 201 failed
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 202 failed
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 203 failed
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 204 failed
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 205 failed
For 3 samples nosel
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 201 failed
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 202 failed
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 203 failed
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 204 failed
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 205 failed
For 4 samples nosel
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 201 failed
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 202 failed
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 203 failed
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 204 failed
22Jul_tests_12Aug/22Jul_6mo nosel 22Jul_6mo_nosel 205 failed
For 5 samples nosel
22Jul

In [12]:
filenames = ["22Jul_deme_1k", "22Jul_deme_5k", "22Jul_deme_20k"]*3
used_filenames = ["22Jul_tests_12Aug"+"/"+f for f in filenames]
prefixes = ["nosel"]*3 + ["lowsel"]*3 + ["highsel"]*3
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 205

for (filename, prefix, matrix_name) in zip(used_filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples nosel
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul_deme_1k_nosel 201 failed
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul_deme_1k_nosel 202 failed
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul_deme_1k_nosel 203 failed
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul_deme_1k_nosel 204 failed
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul_deme_1k_nosel 205 failed
For 3 samples nosel
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul_deme_1k_nosel 201 failed
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul_deme_1k_nosel 202 failed
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul_deme_1k_nosel 203 failed
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul_deme_1k_nosel 204 failed
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul_deme_1k_nosel 205 failed
For 4 samples nosel
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul_deme_1k_nosel 201 failed
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul_deme_1k_nosel 202 failed
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul_deme_1k_nosel 203 failed
22Jul_tests_12Aug/22Jul_deme_1k nosel 22Jul

In [13]:
filenames = ["22Jul_rep_0.25", "22Jul_rep_0.5", "22Jul_rep_0.75"]*3
used_filenames = ["22Jul_tests_12Aug"+"/"+f for f in filenames]
prefixes = ["nosel"]*3 + ["lowsel"]*3 + ["highsel"]*3
matrix_names = [filename+"_"+prefix for filename, prefix in zip(filenames, prefixes)]


max_counts = 205

for (filename, prefix, matrix_name) in zip(used_filenames, prefixes, matrix_names):
    for no_samples in range(2, 9):
        print("For "+str(no_samples) + " samples", prefix)
        stat_matrix = []
        for number in range(1, max_counts+1):
            try:
                stats = calculate_output_stats(filename, prefix, base_filepath, number, no_samples)
                stat_matrix.append(stats)
                #print(stats)
            except:
                print(filename, prefix, matrix_name, number, "failed")
        np.save(base_filepath+"/"+matrix_name+"_"+str(no_samples)+"_samples.npy", stat_matrix)
        

For 2 samples nosel
22Jul_tests_12Aug/22Jul_rep_0.25 nosel 22Jul_rep_0.25_nosel 201 failed
22Jul_tests_12Aug/22Jul_rep_0.25 nosel 22Jul_rep_0.25_nosel 202 failed
22Jul_tests_12Aug/22Jul_rep_0.25 nosel 22Jul_rep_0.25_nosel 203 failed
22Jul_tests_12Aug/22Jul_rep_0.25 nosel 22Jul_rep_0.25_nosel 204 failed
22Jul_tests_12Aug/22Jul_rep_0.25 nosel 22Jul_rep_0.25_nosel 205 failed
For 3 samples nosel
22Jul_tests_12Aug/22Jul_rep_0.25 nosel 22Jul_rep_0.25_nosel 201 failed
22Jul_tests_12Aug/22Jul_rep_0.25 nosel 22Jul_rep_0.25_nosel 202 failed
22Jul_tests_12Aug/22Jul_rep_0.25 nosel 22Jul_rep_0.25_nosel 203 failed
22Jul_tests_12Aug/22Jul_rep_0.25 nosel 22Jul_rep_0.25_nosel 204 failed
22Jul_tests_12Aug/22Jul_rep_0.25 nosel 22Jul_rep_0.25_nosel 205 failed
For 4 samples nosel
22Jul_tests_12Aug/22Jul_rep_0.25 nosel 22Jul_rep_0.25_nosel 201 failed
22Jul_tests_12Aug/22Jul_rep_0.25 nosel 22Jul_rep_0.25_nosel 202 failed
22Jul_tests_12Aug/22Jul_rep_0.25 nosel 22Jul_rep_0.25_nosel 203 failed
22Jul_tests_12Aug

And now for the application to real tumours.

In [ ]:
#read in mutations and correct for clonality
df = pd.read_csv("C:/Users/44749/Documents/PhD_work/datasets_for_analysis/full_tx421primary_muttable_vaf_ccf_clonality.csv")

print("Table loaded")

filtered_df= df[df['PASS'] == True & ((df['Timing_SC'] != "E") & (df['Timing_SC'] != "N"))]

unique_tumours = filtered_df[['cruk_tumour_id']].drop_duplicates()

purity_df = 

#translate table into dictionary

driver_mut_dict = {True:1, False:0}
all_tumour_names = []

for tumour in unique_tumours.values.tolist():
    tID = tumour[0]
    all_drivs_found = []
    all_tumour_names.append(tID)
    df_here = filtered_df[filtered_df['cruk_tumour_id'] == tID]
    unique_regions = df_here[['region']].drop_duplicates()
    #print(unique_regions)
    tumour_regions = [{} for m in range(len(unique_regions.values.tolist()))]
    #print(unique_regions)
    for n, region in enumerate(unique_regions.values.tolist()):
        regID = region[0]
        #print(tID, regID)
        relevant_muts = df_here[df_here['region']==regID]
        #print(len(relevant_muts))
        for index, row in relevant_muts.iterrows():
            mutID = str(row['chr']) + "." +str(row['start']) + "." + str(row['ref']) + "." + str(row['var']) + ":" + str(row['Hugo_Symbol']) + ":" + str(driver_mut_dict[row['DriverMut']])
            ccf = min(1, float(row['final_ccf'])) if row['cluster_clonality'] != 'clonal' else 1
            #print(tID, regID, mutID, ccf)
            #print(regID, ccf)
            if ccf > 0:
                tumour_regions[n][mutID] = ccf
                if row['DriverMut']:
                    all_drivs_found.append(row['Hugo_Symbol']) #add driver to list if it's found here, might be added more than once- record only the Hugo Symbol, NOT the specific mutation
                    #print(mutID)
        # print(tumour_regions[n])
    np.save(tID+"_mutdict.npy", tumour_regions)
    
    np.save(base_filepath+"/"+tID+"_driver_muts.npy", np.unique(all_drivs_found))

np.save("all_tumour_names.npy", all_tumour_names)
    

    

In [ ]:
#define a function that will do this for individual tumours

#calculate output summary statistics for a given number of samples, excluding clonal mutations
def calculate_output_stats_real_tumour(tumour_name):
    mutdict = np.load(tumour_name + "_mutdict.npy", allow_pickle=True)
    no_samples = len(mutdict)
    all_muts = []
    for dict in mutdict:
        all_muts += list(dict.keys()) #look at all mutations seen in this sample
    all_unique_mutations = np.unique(all_muts) #all mutations in all samples
    mut_matrix = np.zeros((len(all_unique_mutations), no_samples))
    for m, mut in enumerate(all_unique_mutations):
        for n, dict in enumerate(mutdict):
            if mut in dict:
                mut_matrix[m][n] = dict[mut] #zero by default
    #now proceed as with a simulated tumour, excluding clonal samples
    min_ccfs = np.min(mut_matrix, axis=1)
    non_clonal_muts = np.where(min_ccfs < 1.0)[0] #not clonal everywhere
    mut_matrix = mut_matrix[non_clonal_muts] #filter out all of the clonal ones
    #now to actually calculate summary statistics!
    #0. all mutations detected subclonally (i.e. with CCF < 1.0 in at least one sample)
    no_muts = len(non_clonal_muts)
    #1. all mutations detected in 1 sample only
    appearances = np.count_nonzero(mut_matrix, axis=1)
    private = np.where(appearances == 1)[0]
    num_private = len(private)
    #2-no_samples, ... number fixed in 1, ... no_samples-1 samples
    fixed_muts = []
    for s in range(1, no_samples):
        fixed_muts.append(count_fixation_number(mut_matrix, s)) #count number fixed in s samples
    #no_samples+1: number of mutations fixed AND private
    private_ccfs = np.max(mut_matrix[private], axis=1) #the maximum CCF of a private mutation will be its CCF in that sample
    fixed_and_private = np.where(private_ccfs == 1.0)[0]
    num_fixed_private = len(fixed_and_private)
    #no_samples + 2-2*no_samples-1- the number of mutations appearing in 2, ... no_samples samples
    num_appearing = []
    for s in range(2, no_samples+1):
        num_appearing.append(len(np.where(appearances == s)[0]))
    return no_samples, [no_muts, num_private] + fixed_muts + [num_fixed_private] + num_appearing    
    

tIDs = np.load("all_tumour_names.npy", allow_pickle=True)

print(len(tIDs))

#assign tumours to a different bucket depending on the number of samples taken
stats_by_sample_number = [[] for s in range(2, 9)]
num_samples = {}
for tumour_name in tIDs:
    try:
        no_samples, stats = calculate_output_stats_real_tumour(tumour_name)
        if no_samples > 1:
            num_samples[tumour_name] = no_samples
            stats_by_sample_number[no_samples-2].append(stats)
            np.save(base_filepath+"/"+tumour_name+"_ss_4424.npy", stats)
            print(tumour_name, "found")
        else:
            print(tumour_name, "too few samples")
            
    except:
        print(tumour_name, "not found")

#print(overall_stats)

for n, stat_matrix in enumerate(stats_by_sample_number):
    np.save("cruk_"+str(n+2)+"_samples_stat.npy", stat_matrix)

np.save("num_samples_by_tumour_name.npy", num_samples)
#np.save("stat_index_by_tumour_name.npy", stat_index_dict)
#np.save("cruk_overall_stats.npy", overall_stats)



Get purity of each sample.

In [6]:
def test(x):
    x += 1
    return x

x=1
for i in range(10):
    test(x)
print(x)

1
